<a href="https://colab.research.google.com/github/Strojove-uceni/23206-final-pose-estimation-for-swing-improvement/blob/main/Yolov7_Pose.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
!git clone https://github.com/Strojove-uceni/23206-final-pose-estimation-for-swing-improvement
!git clone https://github.com/WongKinYiu/yolov7.git
!wget https://github.com/WongKinYiu/yolov7/releases/download/v0.1/yolov7-w6-pose.pt
import torch
import sys
sys.path.append('/content/yolov7')
sys.path.append('/content/23206-final-pose-estimation-for-swing-improvement')
from torchvision import transforms
from utils.datasets import letterbox
from utils.general import non_max_suppression_kpt
from utils.plots import output_to_keypoint, plot_skeleton_kpts

import cv2
import csv
import os
import math
import datetime
import numpy as np
from Yolov7_Pose_class import Yolov7_PoseEstimation

Cloning into '23206-final-pose-estimation-for-swing-improvement'...
remote: Enumerating objects: 4247, done.
remote: Counting objects: 100% (123/123), done.
remote: Compressing objects: 100% (92/92), done.
remote: Total 4247 (delta 65), reused 68 (delta 30), pack-reused 4124
Receiving objects: 100% (4247/4247), 1.45 GiB | 34.01 MiB/s, done.
Resolving deltas: 100% (867/867), done.
Cloning into 'yolov7'...
remote: Enumerating objects: 1197, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 1197 (delta 2), reused 3 (delta 1), pack-reused 1191
Receiving objects: 100% (1197/1197), 74.24 MiB | 28.40 MiB/s, done.
Resolving deltas: 100% (518/518), done.
--2023-12-15 11:59:16--  https://github.com/WongKinYiu/yolov7/releases/download/v0.1/yolov7-w6-pose.pt
Resolving github.com (github.com)... 140.82.114.4
Connecting to github.com (github.com)|140.82.114.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: ht

In [15]:
!unzip /content/23206-final-pose-estimation-for-swing-improvement/cropped_videos.zip
!rm /content/23206-final-pose-estimation-for-swing-improvement/cropped_videos.zip

Archive:  /content/23206-final-pose-estimation-for-swing-improvement/cropped_videos.zip
  inflating: cropped_videos/output_video__qtYF0FJBWo.mp4  
  inflating: cropped_videos/output_video__rFatlaZhEQ.mp4  
  inflating: cropped_videos/output_video_04d08bM6-6U.mp4  
  inflating: cropped_videos/output_video_0PuF8duUo1A.mp4  
  inflating: cropped_videos/output_video_0rliVha14ZQ.mp4  
  inflating: cropped_videos/output_video_10lx1mFw1eY.mp4  
  inflating: cropped_videos/output_video_1a2QrnvBUuI.mp4  
  inflating: cropped_videos/output_video_1OOqvNPDsDA.mp4  
  inflating: cropped_videos/output_video_276Ec8-zeW8.mp4  
  inflating: cropped_videos/output_video_2cD6CPZhiEY.mp4  
  inflating: cropped_videos/output_video_2Tj5jJ1Kh_Q.mp4  
  inflating: cropped_videos/output_video_2Us2Zx_S6hI.mp4  
  inflating: cropped_videos/output_video_37YZm9oCFnU.mp4  
  inflating: cropped_videos/output_video_3dY8wZdR9R8.mp4  
  inflating: cropped_videos/output_video_3nG1ugrwzH8.mp4  
  inflating: cropped_videos

In [16]:
# Set input type
input_type = 'file'

if input_type == 'file':
    # SET AN INPUT FILE in the folder cropped_videos
    input_file = 'cropped_videos/output_video_nFTbqjkDm1k.mp4'
    csv_file_name = f'{os.path.splitext(input_file)[0]}.csv'
    output_video_name = f'{os.path.splitext(input_file)[0]}_output.mp4'

    # Create 'statistics' folder if it doesn't exist
    statistics_folder = 'statistics'
    os.makedirs(statistics_folder, exist_ok=True)

    # Set the paths for csv file and output video in the 'statistics' folder
    csv_file_path = os.path.join(statistics_folder, os.path.basename(csv_file_name))
    output_video_path = os.path.join(statistics_folder, os.path.basename(output_video_name))

    video_processor = Yolov7_PoseEstimation(input_file, csv_file_path, output_video_path)
    video_processor.pose_estimation()

elif input_type == 'folder':
    input_folder = 'cropped_videos'
    output_folder_videos = 'result_videos'
    output_folder_csv = 'result_statistics'

    # Create output folders if they don't exist
    os.makedirs(output_folder_videos, exist_ok=True)
    os.makedirs(output_folder_csv, exist_ok=True)

    # Iterate through files in the input folder
    for filename in os.listdir(input_folder):
        if filename.endswith(".mp4"):
            # Set output file names
            csv_file_name = f'{os.path.splitext(filename)[0]}.csv'
            output_video_name = f'{os.path.splitext(filename)[0]}_output.mp4'
            csv_file_path = os.path.join(output_folder_csv, csv_file_name)
            output_video_path = os.path.join(output_folder_videos, output_video_name)
            file_path = os.path.join(input_folder, filename)

            video_processor = Yolov7_PoseEstimation(file_path, csv_file_path, output_video_path)
            video_processor.pose_estimation()
else:
    print("You have just two options: 'file' or 'folder'")